In [4]:
import os
import numpy as np
import nibabel as nib

### Testing pipeline
The following code was used to generate the dice score, sensitivity and specificity score for all the models.
This is done for all lesion sizes and for the small lesions (Sizes decending between 100mm³ - 10mm³)

The code below shows how this is done for LGA, only the path and name of the images has to be changed to get scores for the other models.

#### Performance metrics

In [5]:
#The three functions below use images in the form of numpy arrays to calcualte the scores
def my_dice(img1,img2):
 intersection = np.logical_and(img1, img2) #
 union = np.logical_or(img1, img2) 
 dice = (2*np.sum(intersection))/(np.sum(union)+np.sum(intersection)) 
 return dice

def sensitivity(pred_image, true_image):
    TP = len(np.where(pred_image + true_image == 2)[0]) 
    FN = len(np.where(pred_image - true_image  == -1)[0]) 
    TPR = TP/(TP + FN) 
    return TPR

def specificity(pred_image, true_image):
    TN = len(np.where(pred_image + true_image == 0)[0]) 
    FP = len(np.where(pred_image - true_image  == 1)[0]) 
    TNR = TN/(TN + FP) 
    return TNR

#### All Lesion scores

In [13]:
path_to_test = '/home/maarhus/Documents/Master/Results/Predictions/LGA/MS_DATA_TEST'

testset = []
for path, dirs, files in os.walk(path_to_test):
    if len(dirs) == 2: 
        testset.append(os.path.basename(path))

In [20]:
#Lists fo save scores
dice_score_list = []
sensitivity_list = []
specificity_list = []

#Name of predicted and true mask
model_predicted_mask = "lga(0.2)_thr_0.1_map.nii"
true_mask = "lesions_binarised.nii"

#Calculate performance metrics for all subjects in testset
for file in testset:
    pred_path = f'{path_to_test}/{file}/{model_predicted_mask}'
    true_path = f'{path_to_test}/{file}/{true_mask}'

    pred_image = nib.load(pred_path).get_fdata()
    mask_image = nib.load(true_path).get_fdata()

    dice_score = round(my_dice(pred_image, mask_image),3)
    sens = round(sensitivity(pred_image, mask_image),3)
    spec = round(specificity(pred_image, mask_image),5)

    print(f'Score of prediction in folder {file} is: {dice_score}, {sens}, {spec}')

    dice_score_list.append(dice_score)
    sensitivity_list.append(sens)
    specificity_list.append(spec)

#Print mean DSC, sensitivity and specificity
print()
mean_score = np.mean(dice_score_list)
std_score = np.std(dice_score_list)

print(f'The mean dice score is: {round(mean_score,3)}')
print(f'The standard deviation of the dice score is: {round(std_score,3)}')

print()
mean_score = np.mean(sensitivity_list)
std_score = np.std(sensitivity_list)

print(f'The mean sensitivity score is: {round(mean_score,3)}')
print(f'The standard deviation of the sensitivity score is: {round(std_score,3)}')

print()
mean_score = np.mean(specificity_list)
std_score = np.std(specificity_list)

print(f'The mean specificity score is: {round(mean_score,5)}')
print(f'The standard deviation of the specificity score is: {round(std_score,5)}')

Score of prediction in folder 2554238985 is: 0.408, 0.275, 0.99999
Score of prediction in folder 1003101869 is: 0.632, 0.704, 0.99967
Score of prediction in folder 6915167302 is: 0.582, 0.516, 0.99994
Score of prediction in folder 1389542280 is: 0.781, 0.823, 0.99986
Score of prediction in folder 2451433951 is: 0.53, 0.581, 0.99998
Score of prediction in folder 8996005906 is: 0.195, 0.148, 0.99996
Score of prediction in folder 9943418622 is: 0.652, 0.709, 0.99986
Score of prediction in folder 1281058682 is: 0.048, 0.03, 0.99997

The mean dice score is: 0.479
The standard deviation of the dice score is: 0.232

The mean sensitivity score is: 0.473
The standard deviation of the sensitivity score is: 0.271

The mean specificity score is: 0.9999
The standard deviation of the specificity score is: 0.0001


#### Small lesion scores

In [17]:
snitt_dice = []
error_dice = []

snitt_sensitivity = []
error_sensitivity = []

snitt_specificity = []
error_specificity = []

path_small_lesions_test = "/home/maarhus/Documents/Master/bwconncomp/LGA/MS_DATA_TEST"
path_small_lesions_true = "/home/maarhus/Documents/Master/bwconncomp/true_test_labels"

#Calculating scores for lesion sizes 100mm³ to 10mm³ for every subject in the test set
for size in range(100, 0, -10):
    dice_score_list = []
    sensitivity_list = []
    specificity_list = []
    
    for file in testset:
        #Images where larger lesions are removed
        pred_path = f'{path_small_lesions_test}/{file}/lesions_removed_size{size}.nii'
        true_path = f'{path_small_lesions_true}/{file}/lesions_removed_size{size}.nii'

        pred_image = nib.load(pred_path).get_fdata()
        mask_image = nib.load(true_path).get_fdata()

        #Calculating scores and adding them to list to calculate mean and std
        dice_score = my_dice(pred_image, mask_image)
        dice_score_list.append(dice_score)

        sens = sensitivity(pred_image, mask_image)
        sensitivity_list.append(sens)

        spec = specificity(pred_image, mask_image)
        specificity_list.append(spec)

    #Calculating mean
    mean_score = np.mean(dice_score_list)
    std_score = np.std(dice_score_list)

    sens_mean = np.mean(sensitivity_list)
    sens_std = np.std(sensitivity_list)

    spec_mean = np.mean(specificity_list)
    spec_std = np.std(specificity_list)

    #Adding mean and std values to list (Rounding to 3 decimals)
    snitt_dice.append(round(mean_score, 3))
    error_dice.append(round(std_score, 3))

    snitt_sensitivity.append(round(sens_mean, 3))
    error_sensitivity.append(round(sens_std, 3))

    snitt_specificity.append(round(spec_mean, 5))
    error_specificity.append(round(spec_std, 5))

#Printing mean scores with std
print(snitt_dice)
print(error_dice)

print()
print(snitt_sensitivity)
print(error_sensitivity)

print()
print(snitt_specificity)
print(error_specificity)

[0.223, 0.228, 0.18, 0.182, 0.161, 0.155, 0.118, 0.089, 0.089, 0.0]
[0.152, 0.147, 0.168, 0.162, 0.141, 0.135, 0.119, 0.12, 0.104, 0.0]

[0.195, 0.197, 0.152, 0.152, 0.138, 0.13, 0.099, 0.074, 0.071, 0.0]
[0.131, 0.125, 0.142, 0.137, 0.12, 0.11, 0.09, 0.093, 0.078, 0.0]

[0.99997, 0.99998, 0.99998, 0.99998, 0.99998, 0.99999, 0.99999, 0.99999, 0.99999, 1.0]
[1e-05, 1e-05, 1e-05, 1e-05, 1e-05, 1e-05, 1e-05, 1e-05, 0.0, 0.0]
